In [1]:
# ch5/porto_seguro’s_safe_driver_prediction_lgb_modeling.ipynb

import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

all_features = all_data.columns.tolist() # 전체 피처

### 결측값 개수를 파생 변수로 생성

In [3]:
# '데이터 하나당 결측값 개수'를 파생 변수로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1)

# 명목형 피처, 태그에 calc가 달린 피처를 제외한 피처
remaining_features = [col for col in all_features \
                      if ('cat' not in col and 'calc' not in col)] 
# num_missing을 remaining_features에 추가
remaining_features.append('num_missing')

### 명목형 피처 원-핫 인코딩 적용

In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [col for col in all_features if 'cat' in col] # 명목형 피처

# 원-핫 인코딩 적용
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

### `ind` 변수의 고유값을 조합한 파생 변수 `mix_ind` 생성

In [5]:
# 태그에 ind가 달린 피처
ind_features = [col for col in all_features if 'ind' in col]

first_col=True
for col in ind_features:
    if first_col:
        all_data['mix_ind'] = all_data[col].astype(str)+'_'
        first_col = False
    else:
        all_data['mix_ind'] += all_data[col].astype(str)+'_'

In [6]:
cat_count_features = []
for col in cat_features+['mix_ind']:
    val_counts_dic = all_data[col].value_counts().to_dict()
    all_data[f'{col}_count'] = all_data[col].apply(lambda x: val_counts_dic[x])
    cat_count_features.append(f'{col}_count')

In [7]:
from scipy import sparse

drop_features = ['ps_ind_14', 'ps_ind_10_bin','ps_ind_11_bin', 
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [8]:
num_train = train.shape[0] # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [9]:
def eval_gini(y_true, y_pred):
    # 실제 값과 예측 값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측 값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)# 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [10]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

## BayesianOptimization을 활용한 하이퍼 파라미터 최적화

In [11]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

# 베이지안 최적화용 데이터 세트
bayes_dtrain = lgbm.Dataset(X_train, y_train)
bayes_dvalid = lgbm.Dataset(X_valid, y_valid)

In [12]:
# 베이지안 최적화를 위한 하이퍼 파라미터 범위
param_bounds = {'num_leaves': (28, 33),
                'lambda_l1': (0.9, 1),
                'lambda_l2': (0.9, 1),
                'feature_fraction': (0.6, 0.8),
                'bagging_fraction': (0.7, 0.9),
                'min_child_samples': (8, 12),
                'min_child_weight': (120, 130)}

# 고정된 하이퍼 파라미터
fixed_params = {'objective': 'binary',
                'learning_rate': 0.01,
                'bagging_freq': 1,
                'verbosity': 0,
                'random_state': 1991}

In [13]:
def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼 파라미터 ---①
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼 파라미터도 추가 ---②
    params.update(fixed_params)
    
    print('하이퍼 파라미터:', params)    
    
    # Light GBM 모델 훈련
    lgb_model = lgbm.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=1500,
                           valid_sets=bayes_dvalid,
                           feval=gini,
                           early_stopping_rounds=150,
                           verbose_eval=False)
    
    best_iter = lgb_model.best_iteration # 최적 반복 횟수
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid, num_iteration=best_iter)
    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수: {gini_score}\n')
    
    return gini_score

In [14]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, # 평가지표 계산 ㅎ마수
                                 pbounds=param_bounds, # 하이퍼 파라미터 범위
                                 random_state=0)

In [15]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=5, n_iter=10)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼 파라미터: {'num_leaves': 30, 'lambda_l1': 0.9602763376071644, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.7430378732744839, 'bagging_fraction': 0.809762700785465, 'min_child_samples': 10, 'min_child_weight': 126.45894113066656, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.01, 'bagging_freq': 1, 'verbosity': 0, 'random_state': 1991}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.145820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
지니계수: 0.2842759041022516

|  1        |  0.2843   |  0.8098   |  0.743    |  0.9603   |  0.9545   |  9.695    |  126.5    |  30.19    |
하이퍼 파라미터: {'num_leaves': 33, 'lambda_l1': 0.

In [16]:
# 평가함수 점수가 최대일 때 하이퍼 파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 9.824601328866194,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28.093949002181777}

In [17]:
# 정수형 하이퍼 파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [18]:
max_params

{'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 10,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28}

In [19]:
# 고정된 하이퍼 파라미터 추가
max_params.update(fixed_params)

In [20]:
max_params

{'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 10,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28,
 'objective': 'binary',
 'learning_rate': 0.01,
 'bagging_freq': 1,
 'verbosity': 0,
 'random_state': 1991}

In [21]:
from sklearn.model_selection import StratifiedKFold

# Stratified K 폴드 교차검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

oof_preds = np.zeros(X.shape[0]) # OOF 예측용 1차원 배열
test_preds = np.zeros(X_test.shape[0]) # 테스트 데이터 예측용 1차원 배열

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 ---①
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # lgbm 데이터세트 생성 ---②
    dtrain = lgbm.Dataset(X_train, y_train) # lgbm 훈련 데이터세트
    dvalid = lgbm.Dataset(X_valid, y_valid) # lgbm 검증 데이터세트

    # Light GBM 모델 훈련 ---③
    lgb_model = lgbm.train(params=max_params, # 최적 하이퍼 파라미터
                           train_set=dtrain, # 훈련 데이터
                           num_boost_round=1500, # 부스팅 반복 횟수
                           valid_sets=dvalid, # 모델 성능 평가용 검증 데이터
                           feval=gini, # 검증용 평가지표
                           early_stopping_rounds=150, # 조기종료 조건
                           verbose_eval=100)
    
    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장 ---④
    best_iter = lgb_model.best_iteration
    # 테스트 데이터를 활용해 예측 ---⑤
    test_preds += lgb_model.predict(X_test, 
                                    num_iteration=best_iter)/folds.n_splits
    # 모델 성능 평가를 위한 oof 예측 ---⑥
    oof_preds[valid_idx] += lgb_model.predict(X_valid, num_iteration=best_iter)
    
    # oof 예측확률에 대한 정규화 지니계수 ---⑦
    gini_score = eval_gini(y_valid, oof_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.140082 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153315	valid_0's gini: 0.270034
[200]	valid_0's binary_logloss: 0.152339	valid_0's gini: 0.280485
[300]	valid_0's binary_logloss: 0.151922	valid_0's gini: 0.286543
[400]	valid_0's binary_logloss: 0.15169	valid_0's gini: 0.291089
[500]	valid_0's binary_logloss: 0.151544	valid_0's gini: 0.294535
[600]	valid_0's binary_logloss: 0.151467	valid_0's gini: 0.296222
[700]	valid_0's binary_logloss: 0.151405	valid_0's gini: 0.297658
[800]	valid_0's binary_logloss: 0.151368	valid_0's gini: 0.298591
[900]	valid_0's binary_logloss: 0.151337	valid_0's gini: 0.299329
[1000]	valid_0's binar

In [22]:
print('OOF 지니계수:', eval_gini(y, oof_preds))

OOF 지니계수: 0.2893420267793303


In [23]:
submission['target'] = test_preds
submission.to_csv('submission.csv')

- 12: num_boost_round = 1400 (pb: 0.29138)
- 13: 0.2893420267793303, num_boost_round = 1500으로 늘림 (0.29144)
- 14: 0.2893420267793303, 재생산 코드
- 15: 재생산 코드
- 16: 베이지안 num_boost_round=1500으로 수정 후 한글화 작업
- 17: 베이지안 출력 로그 빼기

In [24]:
max_params

{'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 10,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28,
 'objective': 'binary',
 'learning_rate': 0.01,
 'bagging_freq': 1,
 'verbosity': 0,
 'random_state': 1991}